# Oracle to Postgres

This notebook demonstrates how to connect to use the `oracle2postgres` package to migrate data between databases. Warning: Running `drop_database` will drop databases. Running `check_for_nulls` with the `remove=True` will remove null characters that appear in strings in the source data.

## Make a backup

We recommend backing up your source and target databases before beginning the migration process.

## Import libraries

In [ ]:
# Import libraries
import sys
import oracle2postgres

## Create logfile

In [ ]:
# create the logfile
oracle2postgres.create_logfile()

## Source database settings

In [ ]:
# get source database settings
source_config = oracle2postgres.get_source_config()

## Target database settings

In [ ]:
# get target database settings
target_config = oracle2postgres.get_target_config()

## Migration settings

In [ ]:
# get settings for migration
migration_config = oracle2postgres.get_migration_config()

## Check the target schema exists

In [ ]:
# check the schema exist on the source database
source_engine = oracle2postgres.connect_to_source(source_config)
oracle2postgres.check_schema_exist(source_engine,source_config['schema_list'])

## Check for null characters in the source database

If null characters exist and they are not removed, your migration will fail

In [ ]:
# check and remove null characters in strings
oracle2postgres.check_for_nulls(source_engine,source_config['schema_list'])

## Remove null characters in the source database

The following command will remove null characters in the source database. Use at your own risk.

In [ ]:
# WARNING: uncomment the following line to remove null characters in the source database
# oracle2postgres.check_for_nulls(source_engine,source_config['schema_list'],remove=True)

## Create the target database

In [ ]:
# connect to the target database
target_engine = oracle2postgres.connect_to_target(target_config)

# WARNING: uncomment the following line to delete target database before creation!
# oracle2postgres.drop_connections(target_config['database'],target_engine)
# oracle2postgres.drop_database(target_config['database'],target_engine)

# create a new database on the target
oracle2postgres.create_database(target_config['database'],target_engine)

## Create the schema on the target database

In [ ]:
# create the schema on the target database
target_engine = oracle2postgres.connect_to_target(target_config,target_config['database'])
oracle2postgres.create_target_schema(source_config['schema_list'],source_engine,target_engine)

## Migrate the data!

In [ ]:
# run the migration
oracle2postgres.migrate(source_config,target_config,migration_config)

## Check the migration

In [ ]:
# run some integrity checks
source_engine = oracle2postgres.connect_to_source(source_config)
target_engine = oracle2postgres.connect_to_target(target_config,target_config['database'])
oracle2postgres.check_migration(source_engine,target_engine,source_config)